In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import pandas as pd
import pickle
from geographiclib.geodesic import Geodesic
import os
import torch

d:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [139]:
X_raw=np.load('D:\Spatial_interpolation\SSIN\data\Te_Hiku/all.npy',allow_pickle=True)

In [140]:
X_raw.shape

(34141, 7, 24)

In [141]:
X_raw=np.array(X_raw)

### X_train

In [142]:
j=6
train_data=X_raw[:,j,:]

In [143]:
train_data.shape

(34141, 24)

In [144]:
train_data = train_data.astype(float)

In [145]:
train_data

array([[19.7, 20.5, 21.4, ..., 21. , 19.9, 19. ],
       [19.7, 20.5, 21.4, ..., 21. , 19.9, 19. ],
       [19.7, 20.5, 21.4, ..., 21. , 19.9, 19. ],
       ...,
       [14.6, 15. , 15.7, ..., 15.5, 14.3, 14.5],
       [14.6, 15. , 15.7, ..., 15.6, 14.3, 14.5],
       [14.6, 15. , 15.8, ..., 15.6, 14.3, 14.5]])

In [146]:
columns_with_missing_data = np.any(np.isnan(train_data), axis=0)
missing_columns = np.where(columns_with_missing_data)[0]
result = np.delete(train_data, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))

In [147]:
missing_columns

array([10], dtype=int64)

In [148]:
result=np.array(result,dtype=float)

In [149]:
result.shape

(34141, 23)

In [150]:
train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
test_list=[16,17,18,19,20,21,22]

In [151]:
train_data=result[:,train_list]
test_data=result[:,test_list]

In [152]:
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)

In [153]:
train_data=np.array(train_data,dtype=float)
test_data=np.array(test_data,dtype=float)

## fusion_data

In [154]:
train_fusion_data=np.delete(X_raw, missing_columns, axis=2)

In [155]:
train_fusion_data=train_fusion_data[:,2:,:]

In [156]:
train_fusion_data=np.array(train_fusion_data,dtype=float)

In [157]:
for feature_index in range(0, train_fusion_data.shape[1]):
    feature_column = train_fusion_data[:, feature_index, :]
    min=np.nanmin(feature_column)
    max=np.nanmax(feature_column)
    norm_data=(feature_column - min) / (max - min)
    train_fusion_data[:, feature_index, :]=norm_data

In [158]:
train_fusion_data[train_fusion_data == 0] = 0.00001
train_fusion_data=np.nan_to_num(train_fusion_data, nan=0)

In [159]:
train_fusion_data.shape

(34141, 5, 23)

In [160]:
train_fusion_data=np.nan_to_num(train_fusion_data, nan=0)

In [161]:
result=train_fusion_data

In [162]:
train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
test_list=[16,17,18,19,20,21,22]

In [163]:
train_fusion_data=result[:,:,train_list]
test_fusion_data=result[:,:,test_list]

In [164]:
train_fusion_data = np.transpose(train_fusion_data, (0, 2, 1))
test_fusion_data = np.transpose(test_fusion_data, (0, 2, 1))

In [165]:
train_fusion_data=np.array(train_fusion_data,dtype=float)
test_fusion_data=np.array(test_fusion_data,dtype=float)

In [166]:
train_fusion_data

array([[[0.02695418, 0.04237288, 0.0880829 , 0.14138817, 0.52531646],
        [0.06469003, 0.10734463, 0.11658031, 0.08997429, 0.57594937],
        [0.        , 0.08474576, 0.12435233, 0.10796915, 0.63291139],
        ...,
        [0.02425876, 0.03954802, 0.17098446, 0.76092545, 0.48734177],
        [0.11051213, 0.09039548, 0.04663212, 0.09254499, 0.55063291],
        [0.01347709, 0.01129944, 0.08290155, 0.22107969, 0.5443038 ]],

       [[0.02695418, 0.04237288, 0.0880829 , 0.14395887, 0.52531646],
        [0.06469003, 0.10734463, 0.11658031, 0.08997429, 0.57594937],
        [0.        , 0.08474576, 0.12176166, 0.10796915, 0.63291139],
        ...,
        [0.02695418, 0.03954802, 0.17098446, 0.76092545, 0.48734177],
        [0.11051213, 0.09039548, 0.04663212, 0.09254499, 0.55063291],
        [0.01347709, 0.01129944, 0.08290155, 0.22107969, 0.5443038 ]],

       [[0.02695418, 0.04237288, 0.0880829 , 0.14395887, 0.52531646],
        [0.06469003, 0.10734463, 0.11658031, 0.08997429, 0.5

### Timestap

In [167]:
timestap_list=[0,1]

In [168]:
timestap=X_raw[:,timestap_list,:]

In [169]:
timestap=timestap.astype(np.str_)

In [170]:
for i in range(len(timestap)):
    for n in range(len(timestap[i][0])):
        timestap[i,0,n]=timestap[i,0,n].replace("-","")
        timestap[i,1,n]=timestap[i,1,n].replace(":","")

In [171]:
timestap=timestap[:,:,1]
timestap_result=[]
for i in range(len(timestap)):
    x=timestap[i,0]+"-"+timestap[i,1]
    timestap_result.append(x)

In [172]:
timestap_result=np.array(timestap_result)

In [173]:
timestap_result.shape

(34141,)

In [174]:
timestap_result

array(['20211209-102000', '20211209-102500', '20211209-103000', ...,
       '20220923-131500', '20220923-132000', '20220923-132500'],
      dtype='<U15')

## invid_mask掩码

In [175]:
def generate1_random_array(length):
    array = []
    for _ in range(length):
        array.append(False)
    return np.array(array)
 
# 生成长度为10的随机数组
train_invid_mask=[]
test_invid_mask=[]
for i in range(len(timestap)):
    random_array = generate1_random_array(len(train_data[0]))
    random_array1 = generate1_random_array(len(test_data[0]))
    train_invid_mask.append(random_array)
    test_invid_mask.append(random_array1)
train_invid_mask=np.array(train_invid_mask)
test_invid_mask=np.array(test_invid_mask)

# test_mask掩码

In [176]:
import random
 
def generate_random_array(length):
    array = []
    for _ in range(length):
            array.append(random.choice([False,True]))
    return np.array(array)

# 生成长度为10的随机数组
test_mask=[]
random_array=generate_random_array(len(test_data[0]))
for i in range(len(timestap)):
    test_mask.append(random_array)
test_mask=np.array(test_mask)

## 地理位置

In [177]:
stations = pd.read_csv("D:\Spatial_interpolation\SSIN\data\Te_Hiku/Tehiku_coordinate.csv")

In [178]:
stations

,Unnamed: 0,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P15,P16,P17,P18,P20,P21,P22,P23,P24,P25
0,Alt,62.606228,69.289428,81.986228,68.399528,78.680578,63.888428,81.039978,60.922928,81.453778,...,66.715378,81.986228,67.662078,67.514528,65.766678,84.923778,75.129478,82.146528,60.111478,59.956428
1,long,173.106350,173.126400,173.142486,173.115715,173.132170,173.118677,173.148900,173.108604,173.146267,...,173.113154,173.142486,173.113248,173.118065,173.119353,173.123860,173.132426,173.133780,173.122335,173.116352
2,Lat,-34.913204,-34.923743,-34.904647,-34.914770,-34.930635,-34.927594,-34.916512,-34.912781,-34.908781,...,-34.922945,-34.904647,-34.924544,-34.923364,-34.922961,-34.918254,-34.913293,-34.913681,-34.928972,-34.929476


In [179]:
stations=stations.T

In [180]:
stations=stations.iloc[1:,1:]

In [181]:
stations = stations.rename(columns={1: 'lon', 2: 'lat'})

In [182]:
stations.shape

(24, 2)

In [183]:
stations

,lon,lat
P1,173.10635,-34.913204
P2,173.1264,-34.923743
P3,173.142486,-34.904647
P4,173.115715,-34.91477
P5,173.13217,-34.930635
P6,173.118677,-34.927594
P7,173.1489,-34.916512
P8,173.108604,-34.912781
P9,173.146267,-34.908781
P10,173.14545,-34.907938


In [184]:
stations.drop([  "P11"],inplace=True)

In [185]:
df_train=stations.iloc[0:16]
df_test=stations.iloc[16:23]

In [186]:
df_test['lat']=df_test['lat'].astype(float)
df_test['lon']=df_test['lon'].astype(float)
df_train['lat']=df_train['lat'].astype(float)
df_train['lon']=df_train['lon'].astype(float)

C:\Users\alienware\AppData\Local\Temp\ipykernel_94892\784921970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['lat']=df_test['lat'].astype(float)
C:\Users\alienware\AppData\Local\Temp\ipykernel_94892\784921970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['lon']=df_test['lon'].astype(float)
C:\Users\alienware\AppData\Local\Temp\ipykernel_94892\784921970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [187]:
def calc_dist_angle_mat(df, out_path):
    lons, lats = df["lon"].values, df["lat"].values
    dist_angle_mat = np.zeros((len(lons), len(lons), 2))

    for i in range(len(lons)):
        for j in range(len(lons)):
            dist = Geodesic.WGS84.Inverse(lats[i], lons[i], lats[j], lons[j])
            dist_angle_mat[i, j, 0] = dist["s12"] / 1000.0  # distance, km
            dist_angle_mat[i, j, 1] = dist["azi1"]  # azimuth at the first point in degrees

    print(dist_angle_mat.shape)
    # print(dist_angle_mat)
    np.save(out_path, dist_angle_mat)

In [188]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    # out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    out_dir = f"{base_dir}"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_train.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_train, out_path)

(16, 16, 2)


In [189]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    # out_dir = f"{base_dir}/BW_132_data"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_test.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_test, out_path)

(7, 7, 2)


In [190]:
def generate_stat_from_train_data(train_info_df, r_pos_mat):
    # info_df = pd.read_csv(info_path)
    # ori_r_pos_mat = np.load(relative_pos_mat_path)

    # is_test = info_df["is_test"].values
    # train_mask = np.where(is_test == 0, True, False)
    # train_info_df = info_df.loc[train_mask, :]

    lat_mean, lat_std, lat_max, lat_min = train_info_df["lat"].mean(), train_info_df["lat"].std(ddof=0), \
                                          train_info_df["lat"].max(), train_info_df["lat"].min()
    lon_mean, lon_std, lon_max, lon_min = train_info_df["lon"].mean(), train_info_df["lon"].std(ddof=0), \
                                          train_info_df["lon"].max(), train_info_df["lon"].min()

    # indexes = np.where(train_mask)[0]
    # idx_i, idx_j = np.ix_(indexes, indexes)
    # r_pos_mat = ori_r_pos_mat[idx_i, idx_j, :]

    r_dist_mat = r_pos_mat[:, :, 0]
    r_angle_mat = r_pos_mat[:, :, 1]

    r_dist_mean, r_dist_std, r_dist_max, r_dist_min = np.mean(r_dist_mat), np.std(r_dist_mat), \
                                                      np.max(r_dist_mat), np.min(r_dist_mat),
    r_angle_mean, r_angle_std, r_angle_max, r_angle_min = np.mean(r_angle_mat), np.std(r_angle_mat), \
                                                          np.max(r_angle_mat), np.min(r_angle_mat)

    stat_dict = {}
    stat_dict["lat_mean"], stat_dict["lat_std"], stat_dict["lat_max"], stat_dict["lat_min"] = \
        lat_mean, lat_std, lat_max, lat_min
    stat_dict["lon_mean"], stat_dict["lon_std"], stat_dict["lon_max"], stat_dict["lon_min"] = \
        lon_mean, lon_std, lon_max, lon_min

    stat_dict["r_dist_mean"], stat_dict["r_dist_std"], stat_dict["r_dist_max"], stat_dict["r_dist_min"] = \
        r_dist_mean, r_dist_std, r_dist_max, r_dist_min
    stat_dict["r_angle_mean"], stat_dict["r_angle_std"], stat_dict["r_angle_max"], stat_dict["r_angle_min"] = \
        r_angle_mean, r_angle_std, r_angle_max, r_angle_min

    print("Calculates the statistics of training data. Done!")

    # with open("./data/hk_data_stats.pkl".format(out_name), "wb") as fp:
    #     pickle.dump(stat_dict, fp)

    return stat_dict

In [191]:
train_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_train.npy")
test_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_test.npy")

In [192]:
stat_dict=generate_stat_from_train_data(df_train, train_df)

Calculates the statistics of training data. Done!


In [193]:
train_df[:, :, 0] = (train_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
train_df[:, :, 1] = (train_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

In [194]:
test_df[:, :, 0] = (test_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
test_df[:, :, 1] = (test_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

### edge_index

In [195]:
def haversine(lat1, lon1, lat2, lon2):
    # 将经纬度从度转换为弧度
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine 公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # 地球半径的平均半径，单位公里
    return km

In [196]:
def compute_distance_matrix(latitudes, longitudes):
    num_sites = len(latitudes)
    distance_matrix = np.zeros((num_sites, num_sites))
    
    for i in range(num_sites):
        for j in range(i + 1, num_sites):
            dist = haversine(latitudes[i], longitudes[i], latitudes[j], longitudes[j])
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist  # 因为距离是对称的

    return distance_matrix

In [197]:
def create_edge_index_from_nearest_neighbors(distance_matrix, num_neighbors=3):
    num_sites = distance_matrix.shape[0]
    edge_index = []

    for i in range(num_sites):
        # 获取与站点 i 距离最近的 num_neighbors 个站点的索引
        neighbors_indices = np.argsort(distance_matrix[i])[:num_neighbors + 1]  # +1 因为最近的包括自己
        for j in neighbors_indices:
            if i != j:  # 避免自连接
                edge_index.append([i, j])

    return torch.tensor(edge_index).t().contiguous()

In [198]:
longitudes_train,latitudes_train=df_train["lon"].values, df_train["lat"].values
# 计算距离矩阵
distance_matrix_train = compute_distance_matrix(longitudes_train, latitudes_train)

# 生成边索引
edge_index_train = create_edge_index_from_nearest_neighbors(distance_matrix_train, num_neighbors=3)

In [199]:
longitudes_test,latitudes_test=df_test["lon"].values, df_test["lat"].values
# 计算距离矩阵
distance_matrix_test = compute_distance_matrix(longitudes_test, latitudes_test)

# 生成边索引
edge_index_test = create_edge_index_from_nearest_neighbors(distance_matrix_test, num_neighbors=3)

## 创建最后的结果

In [200]:
test={"test_data":test_data,"invalid_masks":test_invid_mask,"test_masks":test_mask,"r_pos_mat":test_df,"timestamps":timestap_result,"edg_index":edge_index_test,"fusion_data":test_fusion_data}

In [201]:
train={"train_data":train_data,"invalid_masks":train_invid_mask,"r_pos_mat":train_df,"timestamps":timestap_result,"edg_index":edge_index_train,"fusion_data":train_fusion_data}

In [250]:
# test={"test_data":test["test_data"],"invalid_masks":test["invalid_masks"],"test_masks":test["test_masks"],"r_pos_mat":test["r_pos_mat"],"timestamps":test["timestamps"],"edg_index":test["edg_index"],"fusion_data":test_fusion_data}

In [251]:
# train={"train_data":train["train_data"],"invalid_masks":train["invalid_masks"],"r_pos_mat":train["r_pos_mat"],"timestamps":train["timestamps"],"edg_index":train["edg_index"],"fusion_data":train_fusion_data}

In [341]:
print(list(test.keys()))

['test_data', 'invalid_masks', 'test_masks', 'r_pos_mat', 'timestamps', 'edg_index', 'fusion_data']


In [342]:
test['timestamps'].shape

(34141,)

In [252]:
with open('train.pkl', 'wb') as f:
    pickle.dump(train, f)


with open('test.pkl', 'wb') as f:
    pickle.dump(test, f)

# test

In [257]:
import pickle
import numpy as np
with open('test.pkl', 'rb') as file:
    data = pickle.load(file)

train=data["fusion_data"]
train[0]
train=data["fusion_data"]
train_data_expanded = np.expand_dims(train, axis=-1)
train_data_expanded.shape
train_data_expanded = np.transpose(train_data_expanded, (0, 2, 1,3))
train_data_expanded.shape
train_data=train_data_expanded[:,:,:,:]
edg_index_in=data["edg_index"]

In [258]:
# 找出 NaN 值的位置
nan_indices = np.argwhere(np.isnan(train_data))

if nan_indices.size > 0:
    print("数组中存在 NaN 值，位置如下：")
    print(nan_indices)
else:
    print("数组中不存在 NaN 值")

数组中不存在 NaN 值


In [134]:
train_data.shape

(34141, 5, 7, 1)

In [260]:
import pickle
import numpy as np
with open('test.pkl', 'rb') as file:
    data = pickle.load(file)

train=data["fusion_data"]
train[0]
train=data["fusion_data"]
train_data_expanded = np.expand_dims(train, axis=-1)
train_data_expanded.shape
train_data_expanded = np.transpose(train_data_expanded, (0, 2, 1,3))
train_data_expanded.shape
train_data=train_data_expanded[:,:,:,:]
edg_index_in=data["edg_index"]
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# class ModifiedAFFWithSoftmax(nn.Module):
#     def __init__(self, num_timestamps=34141, num_features=5, r=4):
#         super(ModifiedAFFWithSoftmax, self).__init__()
#         self.num_timestamps = num_timestamps
#         self.num_features = num_features
#         # self.num_sites = num_sites
#         inter_channels = int(num_features // r * num_timestamps)
        
#         # 图卷积网络

#         self.GCN1=GCNConv(num_timestamps * num_features, inter_channels)
#         self.batch1=nn.BatchNorm1d(inter_channels)
#         self.relu1=nn.ReLU(inplace=True)
#         self.GCN2=GCNConv(inter_channels, num_timestamps * num_features)
#         self.batch2=nn.BatchNorm1d(num_timestamps * num_features)
#         # self.relu2=nn.ReLU(inplace=True)
        
#         # 传统卷积部分调整为一维操作，以处理1D特征数据
#         self.global_att = nn.Sequential(
#             nn.AdaptiveAvgPool2d(1),
#             nn.Conv2d(num_features, inter_channels, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(inter_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(inter_channels, num_features, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(num_features),   
#         )

#         self.mlp=nn.Linear(2,1)


#     def forward(self, x, edge_index):
#         # 调整x的形状以适应图卷积网络的输入
#         num_sites=x.size(2)
#         x_gcn=x
#         x_gcn=x_gcn.squeeze(-1).permute(2, 1, 0)
#         x_gcn = x_gcn.reshape(num_sites, -1)  # (103, 8784*9)


#         # GCN 处理
#         x_gcn=self.GCN1(x_gcn, edge_index)
#         x_gcn=self.batch1(x_gcn)
#         x_gcn=self.relu1(x_gcn)
#         x_gcn=self.GCN2(x_gcn, edge_index)
#         xl=self.batch2(x_gcn)
#         # xl=self.relu2(x_gcn)
        
#         # 调整为原始时间戳维度
#         xl = xl.view(num_sites,  self.num_features,self.num_timestamps).permute(2, 1, 0).unsqueeze(-1)  # (8784, 9, 103, 1)

#         # Global attention 处理
#         xg = self.global_att(x)
#         repeat_num=xl.size(2)
#         xg=xg.repeat(1, 1, repeat_num, 1)
#         xlg = torch.cat((xl, xg), dim=3)
#         batch_size, num_channels, height, width = xlg.size()
#         xlg = xlg.view(-1, width)
#         xlg = F.relu(self.mlp(xlg))
#         xlg=xlg.view(batch_size,num_channels,height,-1)

#         weights = xlg.view(xlg.size(0), self.num_features, -1)
#         weights = F.softmax(weights, dim=1)
#         weights = weights.view_as(xlg)
        
#         separated_weights = torch.chunk(weights, self.num_features, dim=1)
#         weighted_features = [x[:, i:i+1, :, :] * separated_weights[i] for i in range(self.num_features)]
        
#         output = torch.sum(torch.stack(weighted_features, dim=0), dim=0)

#         return output.squeeze(1)
# class ModifiedAFFWithSoftmax(nn.Module):
#     def __init__(self, channels=5, r=4, num_features=5):
#         super(ModifiedAFFWithSoftmax, self).__init__()
#         self.layer1 = nn.Linear(5, 1)
#     def forward(self, x):
#         batch_size, num_channels, height, width = x.size()
#         x=x.squeeze(-1)
#         x=x.transpose(1,2)
#         x=x.view(-1,num_channels)
#         x1=self.layer1(x)
#         output=F.relu(x1)
#         output=output.view(batch_size,height,1)

#         return output.squeeze(1)

class ModifiedAFFWithSoftmax(nn.Module):
    def __init__(self, channels=5, r=4, num_features=5):
        super(ModifiedAFFWithSoftmax, self).__init__()
        self.num_features = num_features
        inter_channels = int(channels // r )
        
        
        self.local_att = nn.Sequential(
            nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(channels),
        )
        
        self.global_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(channels),
        )

        self.mlp=nn.Linear(2,1)
    def forward(self, x):
        xl = self.local_att(x)
        xg = self.global_att(x)
        repeat_num=xl.size(2)
        xg=xg.repeat(1, 1, repeat_num, 1)
        xlg = torch.cat((xl, xg), dim=3)
        batch_size, num_channels, height, width = xlg.size()
        xlg = xlg.view(-1, width)
        xlg = F.relu(self.mlp(xlg))
        xlg=xlg.view(
        batch_size,num_channels,height,-1)
        # xlg = xl + xg
        # xlg = xlg - xl
        
        weights = xlg.view(xlg.size(0), self.num_features, -1)
        weights = F.softmax(weights, dim=1)
        weights = weights.view_as(xlg)
        
        separated_weights = torch.chunk(weights, self.num_features, dim=1)
        weighted_features = [x[:, i:i+1, :, :] * separated_weights[i] for i in range(self.num_features)]
        
        output = torch.sum(torch.stack(weighted_features, dim=0), dim=0)

        return output.squeeze(1)
# 初始化模型
model = ModifiedAFFWithSoftmax()

# 生成模拟输入数据
train_tensor = torch.from_numpy(train_data)  # Batch size 8784, 18 features, 41 sites, width 1
train_tensor = train_tensor.float()

# 前向传播
output_tensor = model(train_tensor)

# 打印输出形状
print("Output shape:", output_tensor.shape)

Output shape: torch.Size([34141, 7, 1])


In [209]:
output_tensor

tensor([[[0.1635],
         [0.1896],
         [0.1889],
         ...,
         [0.2917],
         [0.1759],
         [0.1730]],

        [[0.1640],
         [0.1896],
         [0.1883],
         ...,
         [0.2922],
         [0.1759],
         [0.1730]],

        [[0.1640],
         [0.1896],
         [0.1889],
         ...,
         [0.2917],
         [0.1754],
         [0.1730]],

        ...,

        [[0.1642],
         [0.1833],
         [0.2078],
         ...,
         [0.4553],
         [0.2065],
         [0.3088]],

        [[0.1642],
         [0.1828],
         [0.2073],
         ...,
         [0.4544],
         [0.2061],
         [0.3165]],

        [[0.1642],
         [0.1828],
         [0.2086],
         ...,
         [0.4548],
         [0.2075],
         [0.3089]]], grad_fn=<SqueezeBackward1>)

In [205]:
output_tensor[0]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<SelectBackward0>)

In [162]:
train_data

array([[[[0.02695417789757413],
         [0.0646900269541779],
         [0.005390835579514829],
         ...,
         [0.024258760107816715],
         [0.11051212938005392],
         [0.013477088948787075]],

        [[0.042372881355932215],
         [0.10734463276836156],
         [0.06779661016949153],
         ...,
         [0.03954802259887007],
         [0.09039548022598871],
         [0.011299435028248596]],

        [[0.08808290155440412],
         [0.11658031088082903],
         [0.028497409326424878],
         ...,
         [0.17098445595854922],
         [0.04663212435233159],
         [0.0829015544041451]],

        [[0.14138817480719792],
         [0.08997429305912596],
         [0.06426735218508996],
         ...,
         [0.7609254498714653],
         [0.09254498714652953],
         [0.22107969151670953]],

        [[0.5424836601307189],
         [0.5947712418300654],
         [0.522875816993464],
         ...,
         [0.5032679738562092],
         [0.5686274509803922